<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/main/metrics_bug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem with multiclass intersection over union calculation

### 0. Prepare Colab, Define Constants

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#! ls
%cd drive/MyDrive/MachineLearning/
#! git clone https://github.com/emely3h/Geospatial_ML.git
%cd Geospatial_ML
! ls

/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
data_exploration  experiments	     models	   pyproject.toml    scripts
docs		  image_processing   poetry.lock   README.md	     sripts
evaluation	  metrics_bug.ipynb  prepare_data  requirements.txt


In [35]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pickle
from keras.utils import Sequence
from data_exploration.mask_stats import Mask_Stats
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import jaccard_score, precision_score

In [6]:
total_tiles = 11121
train_tiles = 6672
test_val_tiles = 2224
data_path = "../data_colab/256_256"
experiment = "experiment_6"
batch_size = 32
tile_size = 256
step_size = 25

In [15]:
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256), dtype=np.uint8)
true = np.copy(test_split_y)

pred = np.memmap("../models/experiment_3/predictions/pred_test_0.npy", mode="r", shape=(test_val_tiles, 256, 256, 3), dtype=np.float32)
pred = np.copy(pred)
pred = np.argmax(pred, axis=-1).astype(np.uint8)

print(true.shape)
print(np.max(true))
print(np.min(true))
print(type(true[0][0][0]))
print(pred.shape)
print(np.max(pred))
print(np.min(pred))
print(type(pred[0][0][0]))

(2224, 256, 256)
2
0
<class 'numpy.uint8'>
(2224, 256, 256)
2
0
<class 'numpy.uint8'>


#### Option 1: Using skicit-learn

In [20]:
pred_skicit = pred.flatten()
true_skicit = true.flatten()
print(true_skicit.shape)
print(np.max(true_skicit))
print(np.min(true_skicit))
print(type(true_skicit[0]))
print(pred_skicit.shape)
print(np.max(pred_skicit))
print(np.min(pred_skicit))
print(type(pred_skicit[0]))

(145752064,)
2
0
<class 'numpy.uint8'>
(145752064,)
2
0
<class 'numpy.uint8'>


In [36]:
iou_skicit = jaccard_score(true_skicit, pred_skicit, average=None)
print(f'invalid IoU using skicit: {iou_skicit[0]}')
print(f'valid IoU using skicit: {iou_skicit[1]}')
print(f'land IoU using skicit: {iou_skicit[2]}')
print(f'mean IoU using skicit: {iou_skicit.sum()/3}')

precision_skicit = precision_score(true_skicit, pred_skicit, average=None)
print(f'invalid precision using skicit: {precision_skicit[0]}')
print(f'valid precision using skicit: {precision_skicit[1]}')
print(f'land precision using skicit: {precision_skicit[2]}')
print(f'mean precision using skicit: {precision_skicit.sum()/3}')

invalid IoU using skicit: 0.8716894474436456
valid IoU using skicit: 0.9117936513897129
land IoU using skicit: 0.9978916914575914
mean IoU using skicit: 0.9271249300969834
invalid precision using skicit: 0.9617775824921121
valid precision using skicit: 0.9336049351027778
land precision using skicit: 0.99891909286962
mean precision using skicit: 0.9647672034881699


#### Option: using [keras.metrics.IoU](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/IoU)

In [24]:
pred_one_hot = to_categorical(pred, num_classes=3, dtype="uint8")
print(pred_one_hot.shape)
true_one_hot = to_categorical(true, num_classes=3, dtype="uint8")
print(true_one_hot.shape)

(2224, 256, 256, 3)
(2224, 256, 256, 3)


In [27]:
true_invalid = true_one_hot[..., 0].flatten()
pred_invalid = pred_one_hot[..., 0].flatten()
true_valid = true_one_hot[..., 1].flatten()
pred_valid = pred_one_hot[..., 1].flatten()
true_land = true_one_hot[..., 2].flatten()
pred_land = pred_one_hot[..., 2].flatten()

print(true_invalid.shape)
print(pred_invalid.shape)
print(true_valid.shape)
print(pred_valid.shape)
print(true_land.shape)
print(pred_land.shape)
print(np.max(true_invalid))
print(np.min(true_invalid))

(145752064,)
(145752064,)
(145752064,)
(145752064,)
(145752064,)
(145752064,)
1
0


In [37]:
keras_iou_invalid = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])
keras_iou_invalid.update_state(true_invalid, pred_invalid)
print(f'Keras invalid IoU: {keras_iou_invalid.result().numpy()}')

keras_iou_valid = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])
keras_iou_valid.update_state(true_valid, pred_valid)
print(f'Keras valid IoU: {keras_iou_valid.result().numpy()}')

keras_iou_land = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])
keras_iou_land.update_state(true_land, pred_land)
print(f'Keras land IoU: {keras_iou_land.result().numpy()}')

keras_p_invalid = tf.keras.metrics.Precision()
keras_p_invalid.update_state(true_invalid, pred_invalid)
print(f'Precision invalid: {keras_p_invalid.result().numpy()}')

keras_p_valid = tf.keras.metrics.Precision()
keras_p_valid.update_state(true_invalid, pred_invalid)
print(f'Precision valid: {keras_p_valid.result().numpy()}')

keras_p_land = tf.keras.metrics.Precision()
keras_p_land.update_state(true_invalid, pred_invalid)
print(f'Precision land: {keras_p_land.result().numpy()}')

Keras invalid IoU: 0.7874400019645691
Keras valid IoU: 0.791369616985321
Keras land IoU: 0.991915762424469
Precision invalid: 0.9617775678634644
Precision valid: 0.9617775678634644
Precision land: 0.9617775678634644


#### Option 3: using [keras.metrics.OneHotIoU](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/OneHotIoU)

In [32]:
keras_onehot_iou_invalid = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[0])
keras_onehot_iou_invalid.update_state(true_one_hot, pred_one_hot)
print(f'Keras invalid IoU: {keras_onehot_iou_invalid.result().numpy()}')

keras_onehot_iou_valid = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[1])
keras_onehot_iou_valid.update_state(true_one_hot, pred_one_hot)
print(f'Keras valid IoU: {keras_onehot_iou_valid.result().numpy()}')

keras_onehot_iou_land = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[2])
keras_onehot_iou_land.update_state(true_one_hot, pred_one_hot)
print(f'Keras land IoU: {keras_onehot_iou_land.result().numpy()}')

Keras invalid IoU: 0.7874400019645691
Keras valid IoU: 0.791369616985321
Keras land IoU: 0.991915762424469


#### Option 4: using [keras.metrics.TruePositives](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/TruePositives), [keras.metrics.TrueNegatives](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/TrueNegatives), [keras.metrics.FalsePositives](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/FalsePositives), [keras.metrics.FalsePositives](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/FalsePositives),

In [38]:
m_fp = tf.keras.metrics.FalsePositives()
m_fp.update_state(true_invalid, pred_invalid)
fp = m_fp.result().numpy()

m_fn = tf.keras.metrics.FalseNegatives()
m_fn.update_state(true_invalid, pred_invalid)
fn = m_fn.result().numpy()

m_tp = tf.keras.metrics.TruePositives()
m_tp.update_state(true_invalid, pred_invalid)
tp = m_tp.result().numpy()

m_tn = tf.keras.metrics.TrueNegatives()
m_tn.update_state(true_invalid, pred_invalid)
tn = m_tn.result().numpy()

print(f'False Positives: {fp}')
print(f'True Positives: {tp}')
print(f'False Negatives: {fn}')
print(f'True Negatives: {tn}')
print(f'IoU invalid: {(tp) / (tp + fn + fp)}')
print(f'pixel sum: {tp + fp + tn + fn}')
print(f'correct sum: {2224 * 256 * 256}')
print(f'Precision invalid: {tp / (tp + fp)}')

False Positives: 1222720.0
True Positives: 30766884.0
False Negatives: 3306090.0
True Negatives: 110456368.0
IoU invalid: 0.8716893792152405
pixel sum: 145752064.0
correct sum: 145752064
Precision invalid: 0.9617775678634644


In [ ]:
# Problem occures only with IoU not with precision
# Problem only occurs if input has a certain size, the bigger the input data the bigger the differences